In [ ]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.neighbors import KNeighborsClassifier as knn

from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from matplotlib import pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict

# Part 1: One-Hot Encoding

In [ ]:
one_hot=pd.read_csv("one_hot.csv")
one_hot.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jqotw\\one_hot.csv'

Dropping irrelevant columns and making X and Y (yt is for 3-class prediction)

In [ ]:
y=one_hot["high_score"].astype(int)
yt=one_hot["good_bad_average"].astype(int)
X=one_hot.drop(["Unnamed: 0","rate..out.of.5.","high_score","good_bad_average"],axis=1)

y.head()

0    0
1    1
2    0
3    1
4    0
Name: high_score, dtype: int32

Finding optimal K for KNN for binary and 3-class predictions with 5-fold CV

In [ ]:
grade=0
scores=[]
index=0
for i in range(100):

    model1=knn(n_neighbors=i+1,metric='cosine')
    fold =KFold(5)
    score=cross_val_score(model1,X,y,cv=fold)
    if np.mean(score) > grade:
        grade=np.mean(score)
        scores=score
        index= i+1
print(scores,grade, index)

[0.66761364 0.68110795 0.69864961 0.65103056 0.66666667] 0.6730136856302902 47


In [ ]:
grade=0
scores=[]
index=0
for i in range(100):

    model1=knn(n_neighbors=i+1,metric="cosine")
    fold =KFold(5)
    score=cross_val_score(model1,X,yt,cv=fold)
    if np.mean(score) > grade:
        grade=np.mean(score)
        scores=score
        index= i+1
print(scores,grade, index)

[0.53977273 0.53551136 0.50888415 0.45771144 0.48898365] 0.5061726675066228 65


Creating confusion matrix for optimal 3-class KNN

In [ ]:
model1best=knn(n_neighbors=65,metric='cosine')
fold=KFold(5)
ypred=cross_val_predict(model1best,X,yt,cv=fold)
confusionknn1=confusion_matrix(yt,ypred)
print(confusionknn1)

[[1669  641  223]
 [1269  676  272]
 [ 632  438 1217]]


Grid Search for Random Forest Hyperparameters for Binary

In [ ]:
Rf=rf()
params={'n_estimators':list(range(50,500,50)),"max_depth":list(range(10,100,10))}
model2=GridSearchCV(Rf,param_grid=params)
model2.fit(X,y)
print(model2.best_params_)

{'max_depth': 10, 'n_estimators': 450}


Applying best model to generate predictive accuracy scores with 5-fold cross validation

In [ ]:
model2best=rf(max_depth=10,n_estimators=450)
fold=KFold(5)
score=cross_val_score(model2best,X,y,cv=fold)
print(score,np.mean(score))

[0.77414773 0.76775568 0.78464819 0.74129353 0.76545842] 0.7666607102474641


Repeating same process for 3-class prediction

In [ ]:
Rf=rf()
params={'n_estimators':list(range(50,500,50)),"max_depth":list(range(10,100,10))}
model23=GridSearchCV(Rf,param_grid=params)
model23.fit(X,yt)
print(model23.best_params_)

{'max_depth': 40, 'n_estimators': 400}


In [ ]:
model23best=rf(max_depth=40, n_estimators=400)
fold=KFold(5)
score=cross_val_score(model23best,X,yt,cv=fold)
ypred=cross_val_predict(model23best,X,yt,cv=fold)
confusionrf1=confusion_matrix(yt,ypred)
print(score,np.mean(score))
print(confusionrf1)

[0.65411932 0.63352273 0.65174129 0.58706468 0.61051883] 0.6273933700006461
[[1621  639  273]
 [ 799 1005  413]
 [ 199  291 1797]]


Conducting same procedure for ANN

In [ ]:
mlp=MLPClassifier(activation='relu',batch_size=32)
MLP_params={'hidden_layer_sizes':[(8,),(16,),(24,),(32,),(40,),(48,),(56,),(64,),(8,2),(16,2),(24,2),(32,2),(40,2),(48,2),(56,2),(64,2),(8,4),(16,4),(24,4),(32,4),(40,4),(48,4),(56,4),(64,4),(8,8),(16,8),(24,8),(32,8),(40,8),(48,8),(56,8),(64,8),(16,16),(24,16),(32,16),(40,16),(48,16),(56,16),(64,16)],'max_iter':list(range(2,31,2))}
model3=GridSearchCV(mlp,param_grid=MLP_params)
model3.fit(X,y)
print(model3.best_params_)


In [ ]:
model3best=MLPClassifier(hidden_layer_sizes=(40,4),activation='relu',max_iter=12,batch_size=32)
fold =KFold(5)
score=cross_val_score(model3best,X,y,cv=fold)

print(score,np.mean(score))



[0.69673295 0.73934659 0.72707889 0.70931059 0.7050462 ] 0.7155030448407314


In [ ]:
mlp=MLPClassifier(activation='relu',batch_size=32)
MLP_params={'hidden_layer_sizes':[(8,2),(16,2),(24,2),(32,2),(40,2),(48,2),(56,2),(64,2),(8,4),(16,4),(24,4),(32,4),(40,4),(48,4),(56,4),(64,4),(8,8),(16,8),(24,8),(32,8),(40,8),(48,8),(56,8),(64,8),(16,16),(24,16),(32,16),(40,16),(48,16),(56,16),(64,16)],'max_iter':list(range(6,20,2))}
model33=GridSearchCV(mlp,param_grid=MLP_params)
model33.fit(X,yt)
print(model33.best_params_)


In [ ]:
model33best=MLPClassifier(hidden_layer_sizes=(64,2),activation='relu',max_iter=8,batch_size=32)
fold=KFold(5)
score=cross_val_score(model33best,X,yt,cv=fold)
ypred=cross_val_predict(model33best,X,yt,cv=fold)
confusionmlp1=confusion_matrix(yt,ypred)
print(score,np.mean(score))
print(confusionmlp1)

# Part 2 Categorical Embedding

In [ ]:
import categorical_embedder as ce
embedded=pd.read_csv("embedding.csv")
embedded=embedded.sample(frac=1).reset_index(drop=True)
embedded.head()

,Unnamed: 0,restaurant.type,rate..out.of.5.,num.of.ratings,avg.cost..two.people.,online_order,table.booking,cuisines.type,area,high_score,good_bad_average
0,2617,"Casual Dining, Bar",3.9,-0.204020,1.423699,No,No,"North Indian, South Indian, Continental, Ameri...",Brookefield,1,3
1,1086,Casual Dining,3.0,-0.256135,0.019257,Yes,No,"North Indian, Biryani, Chinese, Seafood",Bannerghatta Road,0,1
2,1496,Casual Dining,3.5,-0.272946,0.343359,No,No,"Andhra, Chettinad",Bellandur,0,2
3,4740,Quick Bites,3.6,0.579379,-0.304845,Yes,No,"Bengali, Seafood, Oriya, North Indian, Biryani",Brookefield,1,2
4,1756,Casual Dining,3.8,0.544075,0.451393,Yes,No,"Chinese, Seafood, North Indian, Biryani, Desserts",Electronic City,1,3


In [ ]:
X2=embedded.drop(["Unnamed: 0", "rate..out.of.5.","high_score","good_bad_average"],axis=1)
y2=embedded['high_score']
y2t=embedded['good_bad_average']
X2.head()

,restaurant.type,num.of.ratings,avg.cost..two.people.,online_order,table.booking,cuisines.type,area
0,"Casual Dining, Bar",-0.204020,1.423699,No,No,"North Indian, South Indian, Continental, Ameri...",Brookefield
1,Casual Dining,-0.256135,0.019257,Yes,No,"North Indian, Biryani, Chinese, Seafood",Bannerghatta Road
2,Casual Dining,-0.272946,0.343359,No,No,"Andhra, Chettinad",Bellandur
3,Quick Bites,0.579379,-0.304845,Yes,No,"Bengali, Seafood, Oriya, North Indian, Biryani",Brookefield
4,Casual Dining,0.544075,0.451393,Yes,No,"Chinese, Seafood, North Indian, Biryani, Desserts",Electronic City


Alphabetizing string lists for cuisine type and restaurant type

In [ ]:
X2['online_order']=X2['online_order'].astype(str)
X2['table.booking']=X2['table.booking'].astype(str)
delim=", "
X2.iloc[:,0]=X2.iloc[:,0].apply(lambda x: ", ".join(sorted(x.split(", "))))
X2.iloc[:,5]=X2.iloc[:,5].apply(lambda x: ", ".join(sorted(x.split(", "))))
X2.head()

,restaurant.type,num.of.ratings,avg.cost..two.people.,online_order,table.booking,cuisines.type,area
0,"Bar, Casual Dining",-0.204020,1.423699,No,No,"American, Asian, Continental, North Indian, So...",Brookefield
1,Casual Dining,-0.256135,0.019257,Yes,No,"Biryani, Chinese, North Indian, Seafood",Bannerghatta Road
2,Casual Dining,-0.272946,0.343359,No,No,"Andhra, Chettinad",Bellandur
3,Quick Bites,0.579379,-0.304845,Yes,No,"Bengali, Biryani, North Indian, Oriya, Seafood",Brookefield
4,Casual Dining,0.544075,0.451393,Yes,No,"Biryani, Chinese, Desserts, North Indian, Seafood",Electronic City


Getting label encoding and training Categorical Embedder (note, this will not work if categorical_embedder is not available, can skip to later section with saved embeddings)

In [ ]:
embedding_info=ce.get_embedding_info(X2)
embedding_info

{'restaurant.type': (60, 30),
 'online_order': (2, 1),
 'table.booking': (2, 1),
 'cuisines.type': (1562, 50),
 'area': (30, 15)}

In [ ]:
X_encoded,encoders = ce.get_label_encoded_data(X2)
X_encoded.head()

,restaurant.type,num.of.ratings,avg.cost..two.people.,online_order,table.booking,cuisines.type,area
0,7,-0.204020,1.423699,0,0,23,5
1,23,-0.256135,0.019257,1,0,742,1
2,23,-0.272946,0.343359,0,0,160,3
3,56,0.579379,-0.304845,1,0,481,5
4,23,0.544075,0.451393,1,0,706,8


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded,y2)
embeddings = ce.get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info,
                            is_classification=True, epochs=100,batch_size=256)
X2 = ce.fit_transform(X2, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
X2.head()

  0%|          | 0/5 [00:00<?, ?it/s]

,num.of.ratings,avg.cost..two.people.,restaurant.type_embedding_0,restaurant.type_embedding_1,restaurant.type_embedding_2,restaurant.type_embedding_3,restaurant.type_embedding_4,restaurant.type_embedding_5,restaurant.type_embedding_6,restaurant.type_embedding_7,...,area_embedding_5,area_embedding_6,area_embedding_7,area_embedding_8,area_embedding_9,area_embedding_10,area_embedding_11,area_embedding_12,area_embedding_13,area_embedding_14
0,-0.204020,1.423699,-0.121215,0.035776,-0.020936,0.096638,0.045150,-0.058542,0.039808,0.037537,...,-0.024569,-0.079604,0.060307,-0.078588,0.013658,-0.113637,0.076305,0.048469,0.089857,0.108448
1,-0.256135,0.019257,-0.047634,0.019641,-0.068351,0.001218,0.055020,-0.011208,0.017521,-0.076554,...,0.140774,-0.115712,-0.035858,-0.007736,0.112494,0.131163,-0.102420,0.044345,0.034707,0.073262
2,-0.272946,0.343359,-0.047634,0.019641,-0.068351,0.001218,0.055020,-0.011208,0.017521,-0.076554,...,0.042546,0.081038,0.040569,-0.031403,0.059590,0.000082,0.034225,0.112458,-0.120721,0.084155
3,0.579379,-0.304845,-0.009982,-0.005069,0.001240,-0.020556,0.007415,-0.017788,-0.013858,0.026838,...,-0.024569,-0.079604,0.060307,-0.078588,0.013658,-0.113637,0.076305,0.048469,0.089857,0.108448
4,0.544075,0.451393,-0.047634,0.019641,-0.068351,0.001218,0.055020,-0.011208,0.017521,-0.076554,...,-0.047808,-0.070530,0.025423,-0.031115,-0.100445,-0.048672,-0.039224,-0.045146,-0.153730,-0.018370


saving embeddings so this can be exported to GPU for grid search

In [ ]:
embedded=pd.DataFrame(X2,y2,y2t)
embedded.to_csv("C:\\Users\\jqotw\\ce.csv")

In [ ]:
This part is for google collab

In [ ]:
ce=pd.read_csv("ce.csv")
ce.head()

,num.of.ratings,avg.cost..two.people.,restaurant.type_embedding_0,restaurant.type_embedding_1,restaurant.type_embedding_2,restaurant.type_embedding_3,restaurant.type_embedding_4,restaurant.type_embedding_5,restaurant.type_embedding_6,restaurant.type_embedding_7,...,area_embedding_7,area_embedding_8,area_embedding_9,area_embedding_10,area_embedding_11,area_embedding_12,area_embedding_13,area_embedding_14,high_score,good_bad_average
0,-0.204020,1.423699,-0.121215,0.035776,-0.020936,0.096638,0.045150,-0.058542,0.039808,0.037537,...,0.060307,-0.078588,0.013658,-0.113637,0.076305,0.048469,0.089857,0.108448,1,3
1,-0.256135,0.019257,-0.047634,0.019641,-0.068351,0.001218,0.055020,-0.011208,0.017521,-0.076554,...,-0.035858,-0.007736,0.112494,0.131163,-0.102420,0.044345,0.034707,0.073262,0,1
2,-0.272946,0.343359,-0.047634,0.019641,-0.068351,0.001218,0.055020,-0.011208,0.017521,-0.076554,...,0.040569,-0.031403,0.059590,0.000082,0.034225,0.112458,-0.120721,0.084155,0,2
3,0.579379,-0.304845,-0.009982,-0.005069,0.001240,-0.020556,0.007415,-0.017788,-0.013858,0.026838,...,0.060307,-0.078588,0.013658,-0.113637,0.076305,0.048469,0.089857,0.108448,1,2
4,0.544075,0.451393,-0.047634,0.019641,-0.068351,0.001218,0.055020,-0.011208,0.017521,-0.076554,...,0.025423,-0.031115,-0.100445,-0.048672,-0.039224,-0.045146,-0.153730,-0.018370,1,3


In [ ]:
X2=ce.drop(["high_score",'good_bad_average'],axis=1)
y2=ce['high_score']
y2t=ce['good_bad_average']
X2.head()

,num.of.ratings,avg.cost..two.people.,restaurant.type_embedding_0,restaurant.type_embedding_1,restaurant.type_embedding_2,restaurant.type_embedding_3,restaurant.type_embedding_4,restaurant.type_embedding_5,restaurant.type_embedding_6,restaurant.type_embedding_7,...,area_embedding_5,area_embedding_6,area_embedding_7,area_embedding_8,area_embedding_9,area_embedding_10,area_embedding_11,area_embedding_12,area_embedding_13,area_embedding_14
0,-0.204020,1.423699,-0.121215,0.035776,-0.020936,0.096638,0.045150,-0.058542,0.039808,0.037537,...,-0.024569,-0.079604,0.060307,-0.078588,0.013658,-0.113637,0.076305,0.048469,0.089857,0.108448
1,-0.256135,0.019257,-0.047634,0.019641,-0.068351,0.001218,0.055020,-0.011208,0.017521,-0.076554,...,0.140774,-0.115712,-0.035858,-0.007736,0.112494,0.131163,-0.102420,0.044345,0.034707,0.073262
2,-0.272946,0.343359,-0.047634,0.019641,-0.068351,0.001218,0.055020,-0.011208,0.017521,-0.076554,...,0.042546,0.081038,0.040569,-0.031403,0.059590,0.000082,0.034225,0.112458,-0.120721,0.084155
3,0.579379,-0.304845,-0.009982,-0.005069,0.001240,-0.020556,0.007415,-0.017788,-0.013858,0.026838,...,-0.024569,-0.079604,0.060307,-0.078588,0.013658,-0.113637,0.076305,0.048469,0.089857,0.108448
4,0.544075,0.451393,-0.047634,0.019641,-0.068351,0.001218,0.055020,-0.011208,0.017521,-0.076554,...,-0.047808,-0.070530,0.025423,-0.031115,-0.100445,-0.048672,-0.039224,-0.045146,-0.153730,-0.018370


Repeating experiment 1 using embedding values

In [ ]:
grade=0
scores=[]
index=0
for i in range(100):

    model1=knn(n_neighbors=i+1)
    fold =KFold(5)
    score=cross_val_score(model1,X2,y2,cv=fold)
    if np.mean(score) > grade:
        grade=np.mean(score)
        scores=score
        index= i+1
print(scores,grade, index)

[0.75284091 0.734375   0.73987207 0.73773987 0.73987207] 0.7409399835239387 29


In [ ]:
grade=0
scores=[]
index=0
for i in range(100):

    model1=knn(n_neighbors=i+1)
    fold =KFold(5)
    score=cross_val_score(model1,X2,y2t,cv=fold)
    if np.mean(score) > grade:
        grade=np.mean(score)
        scores=score
        index= i+1
print(scores,grade, index)

[0.57670455 0.546875   0.55010661 0.55934613 0.58066809] 0.5627400739807455 82


In [ ]:
model1best=knn(n_neighbors=82)
fold=KFold(5)
ypred=cross_val_predict(model1best,X2,y2t,cv=fold)
confusionknn2=confusion_matrix(y2t,ypred)
print(confusionknn2)

[[1875  515  143]
 [1285  708  224]
 [ 477  433 1377]]


In [ ]:
Rf=rf()
params={'n_estimators':list(range(50,500,50)),"max_depth":list(range(10,100,10))}
model2=GridSearchCV(Rf,param_grid=params)
model2.fit(X2,y2)
print(model2.best_params_)

{'max_depth': 10, 'n_estimators': 250}


In [ ]:
model2best=rf(max_depth=10,n_estimators=250)
fold=KFold(5)
score=cross_val_score(model2best,X,y,cv=fold)
print(score,np.mean(score))

[0.77627841 0.76491477 0.78535892 0.73987207 0.7604833 ] 0.7653814935064934


In [ ]:
Rf=rf()
params={'n_estimators':list(range(50,500,50)),"max_depth":list(range(10,100,10))}
model23=GridSearchCV(Rf,param_grid=params)
model23.fit(X2,y2t)
print(model23.best_params_)

{'max_depth': 10, 'n_estimators': 450}


In [ ]:
model23best=rf(max_depth=10, n_estimators=450)
fold=KFold(5)
score=cross_val_score(model23best,X,yt,cv=fold)
ypred=cross_val_predict(model23best,X,yt,cv=fold)
confusionrf2=confusion_matrix(yt,ypred)
print(score,np.mean(score))
print(confusionrf2)

[0.62428977 0.59446023 0.62544421 0.54797441 0.56361052] 0.591155828002843
[[2005  182  346]
 [1448  282  487]
 [ 326   92 1869]]


In [ ]:
mlp=MLPClassifier(activation='relu',batch_size=32)
MLP_params={'hidden_layer_sizes':[(8,2),(16,2),(24,2),(32,2),(40,2),(48,2),(56,2),(64,2),(8,4),(16,4),(24,4),(32,4),(40,4),(48,4),(56,4),(64,4),(8,8),(16,8),(24,8),(32,8),(40,8),(48,8),(56,8),(64,8),(16,16),(24,16),(32,16),(40,16),(48,16),(56,16),(64,16)],'max_iter':list(range(6,20,2))}
model3=GridSearchCV(mlp,param_grid=MLP_params)
model3.fit(X2,y2)
print(model3.best_params_)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (6) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (6) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (6) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (6) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686

NameError: name 'model33' is not defined

In [ ]:
print(model3.best_params_)

{'hidden_layer_sizes': (64, 4), 'max_iter': 18}


In [ ]:
model3best=MLPClassifier(hidden_layer_sizes=(64,4),activation='relu',max_iter=20,batch_size=32)
fold =KFold(5)
score=cross_val_score(model3best,X2,y2,cv=fold)

print(score,np.mean(score))


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(


[0.79190341 0.7890625  0.79317697 0.79530917 0.80099502] 0.7940894149382955


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
mlp=MLPClassifier(activation='relu',batch_size=32)
MLP_params={'hidden_layer_sizes':[(8,2),(16,2),(24,2),(32,2),(40,2),(48,2),(56,2),(64,2),(8,4),(16,4),(24,4),(32,4),(40,4),(48,4),(56,4),(64,4),(8,8),(16,8),(24,8),(32,8),(40,8),(48,8),(56,8),(64,8),(16,16),(24,16),(32,16),(40,16),(48,16),(56,16),(64,16)],'max_iter':list(range(6,20,2))}
model33=GridSearchCV(mlp,param_grid=MLP_params)
model33.fit(X2,y2t)
print(model33.best_params_)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (6) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (6) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (6) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (6) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686

{'hidden_layer_sizes': (64, 16), 'max_iter': 18}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
model33best=MLPClassifier(hidden_layer_sizes=(64,16),activation='relu',max_iter=21,batch_size=32)
fold=KFold(5)
score=cross_val_score(model33best,X2,y2t,cv=fold)
ypred=cross_val_predict(model33best,X2,y2t,cv=fold)
confusionmlp2=confusion_matrix(y2t,ypred)
print(score,np.mean(score))
print(confusionmlp2)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (21) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (21) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (21) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (21) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py

[0.59375    0.60085227 0.59488273 0.59914712 0.59772566] 0.5972715561801383
[[1866  473  194]
 [1214  645  358]
 [ 251  291 1745]]


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (21) reached and the optimization hasn't converged yet.
  warnings.warn(
